# Background

E-commerce companies like AMazon , flipkart uses different recommendation systems to provide suggestions to the customers.Amazon uses currently item-item collaberrative filtering, which scales to massive datasets and produces high quality recommendation system in the real time. This system is a kind of a information filtering system which seeks to predict the "rating" or preferences which user is interested in.


## Introduction to Recommendation systems


A recommendation engine is a system or algorithm that analyzes user data and provides personalized suggestions or recommendations for items or content that the user may be interested in. These recommendations are based on various factors such as the user's preferences, historical behavior, demographic information, and similarities to other users.

Recommendation engines are commonly used in e-commerce, OTT plateforms , social media, and other online services to enhance user experience and engagement. They help users discover new products, movies, music, articles, or any other items that align with their interests.

Recommeder system creates a similarity between the user and items and exploits the similarity between user/item to make recommendations.

## What recommeder system can solve ?

1.  It can help the user to find the right product.
2.  It can increase the user engagement. For example, there's 40% more click on the google news due to recommendation.
3.  It helps the item providers to deliver the items to the right user.In Amazon , 35 % products get sold due to recommendation.
4.  It helps to make the contents more personalized.In Netflix most of the rented movies are from recommendations.

## Types of recommendations

There are mainly 6 types of the recommendations systems :-

1.  Popularity based systems :- It works by recommeding items viewed and purchased by most people and are rated high.It is not a personalized recommendation.
2.  Classification model based:- It works by understanding the features of the user and applying the classification algorithm to decide whether the user is interested or not in the product.
3. Content based recommedations:- It is based on the information on the contents of the item rather than on the user opinions.The main idea is if the user likes an item then he or she will like the "other" similar item.
4. Collaberative Filtering:- It is based on assumption that people like things similar to other things they like, and things that are liked by other people with similar taste. it is mainly of two types: a) User-User b) Item -Item

5. Hybrid Approaches:- This system approach is to combine collaborative filtering, content-based filtering, and other approaches .

6. Association rule mining :- Association rules capture the relationships between items based on their patterns of co-occurrence across transactions


## Attribute Information:

● userId : Every user identified with a unique id

● productId : Every product identified with a unique id

● Rating : Rating of the corresponding product by the corresponding user

● timestamp : Time of the rating ( ignore this column for this exercise)

# 1. Fetching data 

## 1.1 Exploratory data analysis

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import os
import importlib
warnings.filterwarnings('ignore')
sns.set_theme(color_codes=True)

In [ ]:
# user-defined function to check library is installed or not, if not installed then it will install automatically at runtime.
def check_and_install_library(library_name):
  try:
    importlib.import_module(library_name)
    print(f"{library_name} is already installed")
  except ImportError:
    print(f"{library_name} is not installed. Installing...")
    try:
      import pip
      pip.main(['install',library_name])
    except:
      print("Error: Failed to install the library.\nPlease install it manually.")

In [ ]:
data = pd.read_csv('./DATA/ratings_Electronics.csv',names=["userId","productId","rating","timestamp"])

In [ ]:
print(data.shape)
print(data.columns)

Since our dataset is too big and it will be difficult to analyze the entire dataset due to limited resources,thats'why I am randomly taking 20% of the data as sample out of the whole dataset which is 1564896.

In [ ]:
electronic_data = data.sample(n=1564896,ignore_index=True)
del data
electronic_data.head()

In [ ]:
print(electronic_data.info())
print(electronic_data.describe())
print(electronic_data.isnull().sum())

In [ ]:
electronic_data[electronic_data.duplicated()].shape[0]

In [ ]:
electronic_data.head()

In [ ]:
plt.figure(figsize=(8,4))
sns.countplot(x='rating',data=electronic_data)
plt.title('Rating Distribution')
plt.xlabel('Rating')
plt.ylabel('Count')
plt.grid()
plt.show()

Most of the user have given 5 rating

In [ ]:
print('Totak rating : ',electronic_data.shape[0])
print('Total unique users : ',electronic_data['userId'].unique().shape[0])
print('Total unique products : ',electronic_data['productId'].unique().shape[0])

#### Analysing the ratinf by user

In [ ]:
no_of_rated_products_per_user = electronic_data.groupby(by='userId')['rating'].count().sort_values(ascending=False)
no_of_rated_products_per_user.head()
print('No of rated product more than 50 per user : {} '.format(sum(no_of_rated_products_per_user >= 50)))

# 2 Popularity Based Recommendation

Popularity based recommendation system works with the trend. It basically uses the items which are in trend right now. For example, if any product which is usually bought by every new user then there are chances that it may suggest that item to the user who just signed up.

The problems with popularity based recommendation system is that the personalization is not available with this method i.e. even though you know the behaviour of the user but you cannot recommend items accordingly.

In [ ]:
data = electronic_data.groupby('productId').filter(lambda x:x['rating'].count()>50)
data.head()

In [ ]:
data = data.drop('timestamp',axis=1)
data.head()

In [ ]:
no_of_rating_per_product = data.groupby('productId')['rating'].count().sort_values(ascending=False)
no_of_rating_per_product.head()

In [ ]:
# top 20 products
no_of_rating_per_product.head(20).plot(kind='bar')
plt.xlabel('Product ID')
plt.ylabel('num of rating')
plt.title('Top 20 product')
# plt.xticks(rotation=90)
plt.show()

In [ ]:
mean_rating_product_count = pd.DataFrame(data.groupby('productId')['rating'].mean())
mean_rating_product_count.head()

In [ ]:
# plot the rating distribution of average rating product
plt.hist(mean_rating_product_count['rating'],bins=100)
plt.title('Mean Rating Distribution')
plt.show()

In [ ]:
# check the skewness of the mean rating data
mean_rating_product_count['rating'].skew()

In [ ]:
mean_rating_product_count['rating_counts'] = pd.DataFrame(data.groupby('productId')['rating'].count())
mean_rating_product_count.head()

In [ ]:
# highest mean rating product
mean_rating_product_count[mean_rating_product_count['rating_counts']==mean_rating_product_count['rating_counts'].max()]

In [ ]:
#min mean rating product
print('min average rating product : ',mean_rating_product_count['rating_counts'].min())
print('total min average rating products : ',mean_rating_product_count[mean_rating_product_count['rating_counts']==mean_rating_product_count['rating_counts'].min()].shape[0])

In [ ]:
#plot the rating count of mean_rating_product_count
plt.hist(mean_rating_product_count['rating_counts'],bins=100)
plt.title('rating count distribution')
plt.show()

In [ ]:
#joint plot of rating and rating counts
sns.jointplot(x='rating',y='rating_counts',data=mean_rating_product_count)
plt.title('Joint Plot of rating and rating counts')
plt.tight_layout()
plt.show()

In [ ]:
plt.scatter(x=mean_rating_product_count['rating'],y=mean_rating_product_count['rating_counts'])
plt.show()

In [ ]:
print('Correlation between Rating and Rating Counts is : {} '.format(mean_rating_product_count['rating'].corr(mean_rating_product_count['rating_counts'])))

# 3. Collaberative filtering (Item-Item recommedation)

Collaborative filtering is commonly used for recommender systems. These techniques aim to fill in the missing entries of a user-item association matrix. We are going to use collaborative filtering (CF) approach. CF is based on the idea that the best recommendations come from people who have similar tastes. In other words, it uses historical item ratings of like-minded people to predict how someone would rate an item.Collaborative filtering has two sub-categories that are generally called memory based and model-based approaches.

In [ ]:
check_and_install_library('surprise')

In [ ]:
from surprise import KNNWithMeans